In [1]:
from music21 import *
import glob
from tqdm import tqdm
import numpy as np
import random
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model 
from sklearn.model_selection import train_test_split

In [2]:
def read_files(file):
  notes=[]
  notes_to_parse=None
  #parse the midi file
  midi=converter.parse(file)
  #seperate all instruments from the file
  instrmt=instrument.partitionByInstrument(midi)

  for part in instrmt.parts:
    #fetch data only of Piano instrument
    if 'Piano' in str(part):
      notes_to_parse=part.recurse()

      #iterate over all the parts of sub stream elements
      #check if element's type is Note or chord
      #if it is chord split them into notes
      for element in notes_to_parse:
        if type(element)==note.Note:
          notes.append(str(element.pitch))
        elif type(element)==chord.Chord:
          notes.append('.'.join(str(n) for n in element.normalOrder))

  #return the list of notes
  return notes

#retrieve paths recursively from inside the directories/files
file_path=["grieg"]
all_files=glob.glob('All Midi Files/'+file_path[0]+'/*.mid',recursive=True)

#reading each midi file
notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)])

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:12<00:00,  1.30it/s]
C:\Users\Madhav Gupta\AppData\Local\Temp\ipykernel_5112\2771216531.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  notes_array = np.array([read_files(i) for i in tqdm(all_files,position=0,leave=True)])


In [3]:
#unique notes
notess = sum(notes_array,[]) 
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

#notes with their frequency
freq=dict(map(lambda x: (x,notess.count(x)),unique_notes))

#get the threshold frequency
print("\nFrequency notes")
for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))

#filter notes greater than threshold i.e. 50
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

Unique Notes: 248

Frequency notes
30 : 116
50 : 89
70 : 64
90 : 51


In [4]:
#timestep
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    #input will be the current index + timestep
    #output will be the next index after timestep
    inp=i[j:j+timesteps] ; out=i[j+timesteps]

    #append the index value of respective notes 
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])

x_new=np.array(x) 
y_new=np.array(y)

In [5]:
#reshape input and output for the model
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#split the input and value into training and testing sets
#80% for training and 20% for testing sets
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2,random_state=42)

In [6]:
#create the model
model = Sequential()
#create two stacked LSTM layer with the latent dimension of 256
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 256)           264192    
                                                                 
 dropout (Dropout)           (None, 50, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 89)                22873     
                                                                 
Total params: 878,169
Trainable params: 878,169
Non-trai

In [7]:
#compile the model using Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

#train the model on training sets and validate on testing sets
model.fit(
    x_train,y_train,
    batch_size=256,epochs=74, 
    validation_data=(x_test,y_test))

Epoch 1/74
35/35 [==============================] - 59s 1s/step - loss: 4.3593 - accuracy: 0.0311 - val_loss: 4.3247 - val_accuracy: 0.0265
Epoch 2/74
35/35 [==============================] - 48s 1s/step - loss: 4.2647 - accuracy: 0.0375 - val_loss: 4.2524 - val_accuracy: 0.0352
Epoch 3/74
35/35 [==============================] - 50s 1s/step - loss: 4.2201 - accuracy: 0.0442 - val_loss: 4.2353 - val_accuracy: 0.0467
Epoch 4/74
35/35 [==============================] - 53s 2s/step - loss: 4.1624 - accuracy: 0.0560 - val_loss: 4.1770 - val_accuracy: 0.0554
Epoch 5/74
35/35 [==============================] - 54s 2s/step - loss: 4.0922 - accuracy: 0.0649 - val_loss: 4.1585 - val_accuracy: 0.0636
Epoch 6/74
35/35 [==============================] - 57s 2s/step - loss: 4.0236 - accuracy: 0.0741 - val_loss: 4.0198 - val_accuracy: 0.0792
Epoch 7/74
35/35 [==============================] - 60s 2s/step - loss: 3.9102 - accuracy: 0.0897 - val_loss: 3.9580 - val_accuracy: 0.0860
Epoch 8/74
35/35 [==

35/35 [==============================] - 57s 2s/step - loss: 0.3700 - accuracy: 0.8831 - val_loss: 2.9554 - val_accuracy: 0.4924
Epoch 60/74
35/35 [==============================] - 56s 2s/step - loss: 0.3623 - accuracy: 0.8896 - val_loss: 2.9719 - val_accuracy: 0.4879
Epoch 61/74
35/35 [==============================] - 31113s 915s/step - loss: 0.3459 - accuracy: 0.8909 - val_loss: 3.0096 - val_accuracy: 0.4851
Epoch 62/74
35/35 [==============================] - 39s 1s/step - loss: 0.3314 - accuracy: 0.8992 - val_loss: 3.0318 - val_accuracy: 0.4943
Epoch 63/74
35/35 [==============================] - 46s 1s/step - loss: 0.3240 - accuracy: 0.8987 - val_loss: 3.0502 - val_accuracy: 0.5016
Epoch 64/74
35/35 [==============================] - 50s 1s/step - loss: 0.3102 - accuracy: 0.9055 - val_loss: 3.0746 - val_accuracy: 0.4938
Epoch 65/74
35/35 [==============================] - 50s 1s/step - loss: 0.3179 - accuracy: 0.9020 - val_loss: 3.0798 - val_accuracy: 0.4998
Epoch 66/74
35/35 [=

In [8]:
#save the model for predictions
model.save("MOD")

INFO:tensorflow:Assets written to: MOD\assets


INFO:tensorflow:Assets written to: MOD\assets


In [9]:
#load the model
model=load_model("MOD")
#generate random index
index = np.random.randint(0,len(x_test)-1)
#get the data of generated index from x_test
music_pattern = x_test[index]

out_pred=[] #it will store predicted notes

#iterate till 200 note is generated
for i in range(200):

  #reshape the music pattern 
  music_pattern = music_pattern.reshape(1,len(music_pattern),1)
  
  #get the maximum probability value from the predicted output
  pred_index = np.argmax(model.predict(music_pattern))
  #get the note using predicted index and
  #append to the output prediction list
  out_pred.append(ind2note[pred_index])
  music_pattern = np.append(music_pattern,pred_index)
  
  #update the music pattern with one timestep ahead
  music_pattern = music_pattern[1:]

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 26ms/step


In [10]:
output_notes = []
for offset,pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
        i_curr_note=int(current_note)
        #cast the current note to Note object and
        #append the current note 
        new_note = note.Note(i_curr_note)
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)
    
    #cast the current note to Chord object
    #offset will be 1 step ahead from the previous note
    #as it will prevent notes to stack up 
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)
  
  else:
    #cast the pattern to Note object apply the offset and 
    #append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#save the midi file 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='AI_composed_music.mid')

'AI_composed_music.mid'